 # ASSIGNMENT-2
### Name-NIKHIL KUMAR
### Roll no-2301560045 

# 1. Deduplication data

In [ ]:
import pandas as pd
import dedupe
import os
customers = pd.read_csv("C:\\Users\\dholi\\OneDrive\\Desktop\\datasets\\customer_data_duped.csv", encoding='utf-8')

In [ ]:
customers.head()
customers.dtypes
for col in customers.columns:
    print(col, customers[col].isnull().sum())

In [ ]:
variables = [
    {'field': 'name', 'type': 'String'},
    {'field': 'job', 'type': 'String'},
    {'field': 'company', 'type': 'String'},  
    {'field': 'street_address','type': 'String'},
    {'field': 'city','type': 'String'},
    {'field': 'state', 'type': 'String', 'has_missing': True},
    {'field': 'email', 'type': 'String', 'has_missing': True},
    {'field': 'user_name', 'type': 'String'},
]

deduper = dedupe.Dedupe(variables)

In [ ]:
deduper

In [ ]:
customers.shape

In [ ]:
deduper.sample(customers.T.to_dict(), 500)

In [ ]:
training_file = '../data/ignore-dedupe-training.json'
#if os.path.exists(training_file):
#    with open(training_file, 'rb') as f:
#        deduper.readTraining(f)

In [ ]:
dedupe.consoleLabel(deduper)

In [ ]:
deduper.train()

In [ ]:
with open(training_file, 'w') as tf:
    deduper.writeTraining(tf)

In [ ]:
dupes = deduper.match(customers.T.to_dict())

In [ ]:
dupes

In [ ]:
dupes[2]

In [ ]:
customers.iloc[[741,1107]]

### Exercise: Flag duplicates by adding 2 extra columns, one for confidence score and one for duplicate_ids

In [ ]:
import numpy as np

dupe_dict = {}

for dupepair, confidence in dupes:
    dupe_dict[dupepair[0]] = {'pair': dupepair, 'confidence': confidence[0]}
    dupe_dict[dupepair[1]] = {'pair': dupepair, 'confidence': confidence[0]}

customers['duplicate_pair'] = customers.index.map(lambda i: dupe_dict[i].get('pair')
                                                  if i in dupe_dict else np.nan)
customers['confidence'] = customers.index.map(lambda i: dupe_dict[i].get('confidence')
                                              if i in dupe_dict else np.nan)

# 2.String matching

In [ ]:
from fuzzywuzzy import fuzz, process

In [ ]:
berlin = ['Berlin, Germany', 
          'Berlin, Deutschland', 
          'Berlin', 
          'Berlin, DE']

In [ ]:
fuzz.partial_ratio(berlin[0], berlin[1])

In [ ]:
fuzz.ratio?

In [ ]:
fuzz.ratio(berlin[0], berlin[1])

In [ ]:
fuzz.token_set_ratio(berlin[0], berlin[1])

In [ ]:
fuzz.token_sort_ratio(berlin[0], berlin[1])

In [ ]:
fuzz.partial_ratio(berlin[1], berlin[2])

In [ ]:
fuzz.ratio(berlin[1], berlin[2])

In [ ]:
fuzz.token_sort_ratio(berlin[1], berlin[2])

In [ ]:
fuzz.token_set_ratio(berlin[2], berlin[3])

In [ ]:
choices = ['Germany', 'Deutschland', 'France', 
           'United Kingdom', 'Great Britain', 
           'United States']

In [ ]:
process.extract('DE', choices, limit=2)

In [ ]:
process.extract('UK', choices)

In [ ]:
process.extract('frankreich', choices)

In [ ]:
process.extract('U.S.', choices)

# 3.Managing Nulls

In [1]:
import pandas as pd
from numpy import random

In [2]:
df = pd.read_csv("C:\\Users\\dholi\\OneDrive\\Desktop\\datasets\\iot_example_with_nulls.csv")

### Data Quality Check

In [3]:
df.head()

,timestamp,username,temperature,heartrate,build,latest,note
0,2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
1,2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2,2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
3,2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
4,2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,NaN,NaN


In [4]:
df.dtypes

timestamp       object
username        object
temperature    float64
heartrate        int64
build           object
latest         float64
note            object
dtype: object

In [5]:
df.note.value_counts()

note
wake        16496
user        16416
interval    16274
sleep       16226
update      16213
test        16068
Name: count, dtype: int64

### Let's remove all null values (including the note: n/a)

In [6]:
df = pd.read_csv("C:\\Users\\dholi\\OneDrive\\Desktop\\datasets\\iot_example_with_nulls.csv", na_values=['n/a'])

### Test to see if we can use dropna

In [7]:
df.shape

(146397, 7)

In [8]:
df.dropna().shape

(46116, 7)

In [9]:
df.dropna(how='all', axis=1).shape

(146397, 7)

### Test to see if we can drop columns

In [10]:
my_columns = list(df.columns)

In [11]:
my_columns

['timestamp',
 'username',
 'temperature',
 'heartrate',
 'build',
 'latest',
 'note']

In [12]:
list(df.dropna(thresh=int(df.shape[0] * .9), axis=1).columns)

['timestamp', 'username', 'heartrate']

### I want to find all columns that have missing data

In [13]:
missing_info = list(df.columns[df.isnull().any()])

In [14]:
missing_info

['temperature', 'build', 'latest', 'note']

In [15]:
for col in missing_info:
    num_missing = df[df[col].isnull() == True].shape[0]
    print('number missing for column {}: {}'.format(col, 
                                                    num_missing))

number missing for column temperature: 32357
number missing for column build: 32350
number missing for column latest: 32298
number missing for column note: 48704


In [16]:
for col in missing_info:
    percent_missing = df[df[col].isnull() == True].shape[0] / df.shape[0]
    print('percent missing for column {}: {}'.format(
        col, percent_missing))

percent missing for column temperature: 0.22102228870810195
percent missing for column build: 0.22097447352063226
percent missing for column latest: 0.22061927498514314
percent missing for column note: 0.332684412931959


### Can I easily substitute majority values in for missing data? 

In [17]:
df.note.value_counts()

note
wake        16496
user        16416
interval    16274
sleep       16226
update      16213
test        16068
Name: count, dtype: int64

In [18]:
df.build.value_counts().head()

build
4e6a7805-8faa-2768-6ef6-eb3198b483ac    1
12aefc6b-272c-751e-6117-134ee73e2649    1
fd4049c3-2297-14ac-a27e-6da57129dd10    1
0bcfab8f-bc25-3f8f-8585-0614e1555fd1    1
b0de05dd-2860-abbb-8be6-f5c0e30ca063    1
Name: count, dtype: int64

In [19]:
df.latest.value_counts()

latest
0.0    75735
1.0    38364
Name: count, dtype: int64

In [20]:
df.latest = df.latest.fillna(0)

### Have not yet addressed temperature missing values... Let's find a way to fill 

In [21]:
df.username.value_counts().head()

username
esmith    45
zsmith    43
vsmith    41
ysmith    40
jsmith    37
Name: count, dtype: int64

In [22]:
df = df.set_index('timestamp')

In [23]:
df.head()

,username,temperature,heartrate,build,latest,note
timestamp,,,,,,
2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,0.0,NaN


In [24]:
df.temperature = df.groupby('username').temperature.fillna(
    method='backfill', limit=3)

### Exercise: How many temperature values did I fill? What percentage of values are still missing (for temperature)?

In [ ]:
rows_filled = 32357 - df[df.temperature.isnull() == True].shape[0] 
still_missing = df[df.temperature.isnull() == True].shape[0] / df.shape[0]

# 4.Scikit Learn Preprocessing

In [2]:
from sklearn import preprocessing
import pandas as pd
from datetime import datetime

In [3]:
hvac = pd.read_csv("C:\\Users\\dholi\\OneDrive\\Desktop\\datasets\\HVAC_with_nulls.csv")

### Checking Data Quality

In [8]:
hvac.dtypes

Date           object
Time           object
TargetTemp    float64
ActualTemp      int64
System          int64
SystemAge     float64
BuildingID      int64
10            float64
dtype: object

In [9]:
hvac.shape

(8000, 8)

In [10]:
hvac.head()

,Date,Time,TargetTemp,ActualTemp,System,SystemAge,BuildingID,10
0,6/1/13,0:00:01,66.0,58,13,20.0,4,NaN
1,6/2/13,1:00:01,NaN,68,3,20.0,17,NaN
2,6/3/13,2:00:01,70.0,73,17,20.0,18,NaN
3,6/4/13,3:00:01,67.0,63,2,NaN,15,NaN
4,6/5/13,4:00:01,68.0,74,16,9.0,3,NaN


### Impute missing values with mean

In [15]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values='NaN', strategy='mean')

In [16]:
hvac_numeric = hvac[['TargetTemp', 'SystemAge']]

In [ ]:
imp = imp.fit(hvac_numeric.loc[:10])

In [ ]:
transformed = imp.fit_transform(hvac_numeric)
transformed

In [ ]:
hvac['TargetTemp'], hvac['SystemAge'] = transformed[:,0], transformed[:,1]

In [20]:
hvac.head()

,Date,Time,TargetTemp,ActualTemp,System,SystemAge,BuildingID,10
0,6/1/13,0:00:01,66.0,58,13,20.0,4,NaN
1,6/2/13,1:00:01,NaN,68,3,20.0,17,NaN
2,6/3/13,2:00:01,70.0,73,17,20.0,18,NaN
3,6/4/13,3:00:01,67.0,63,2,NaN,15,NaN
4,6/5/13,4:00:01,68.0,74,16,9.0,3,NaN


### Scale temperature values

In [22]:
hvac['ScaledTemp'] = preprocessing.scale(hvac['ActualTemp'])

In [23]:
hvac['ScaledTemp'].head()

0   -1.293272
1    0.048732
2    0.719733
3   -0.622270
4    0.853934
Name: ScaledTemp, dtype: float64

### Scale using a min and max scaler 

In [24]:
min_max_scaler = preprocessing.MinMaxScaler()

In [25]:
temp_minmax = min_max_scaler.fit_transform(hvac[['ActualTemp']])

In [26]:
temp_minmax

array([[0.12],
       [0.52],
       [0.72],
       ...,
       [0.56],
       [0.32],
       [0.44]])

### Exercise: add the temp_minmax back to the dataframe as a new column

In [27]:
hvac['MinMaxScaledTemp'] = temp_minmax[:,0]
hvac['MinMaxScaledTemp'].head()

0    0.12
1    0.52
2    0.72
3    0.32
4    0.76
Name: MinMaxScaledTemp, dtype: float64